> **Note:** In most sessions you will be solving exercises posed in a Jupyter notebook that looks like this one. Because you are cloning a Github repository that only we can push to, you should **NEVER EDIT** any of the files you pull from Github. Instead, what you should do, is either make a new notebook and write your solutions in there, or **make a copy of this notebook and save it somewhere else** on your computer, not inside the `sds` folder that you cloned, so you can write your answers in there. If you edit the notebook you pulled from Github, those edits (possible your solutions to the exercises) may be overwritten and lost the next time you pull from Github. This is important, so don't hesitate to ask if it is unclear.

In [1]:
import scraping_class
logfile = 'log.txt' ## name your log file.
connector = scraping_class.Connector(logfile)

# Exercise Set 10: Parsing and Information Extraction

In this Exercise Set we shall develop our webscraping skills even further by practicing **parsing** and navigating html trees using BeautifoulSoup and furthermore train extracting information from raw text with no html tags to help, using regular expressions. 

But just as importantly you will get a chance to think about **data quality issues** and how to ensure reliability when curating your own webdata. 

## Exercise Section 10.1: Logging and data quality

> **Ex. 10.1.1:** *Why is is it important to log processes in your data collection?*

** answer goes here** 

** solution** When designing your own data collection you are in charge of ensuring the quality. Many processes can go wrong, which if not spotted can lead to fundamental distortions of your dataset and in turn conclusions. 

> **Ex. 10.1.2:**
*How does logging help with both ensuring and documenting the quality of your data?*
** answer goes here** 

** solution** Having a log allow you to spot the potential anomalies in the website you are scraping. Examples could be errors and failed calls not randomly distributed accross time or subdomains. This would prompt you to investigate the issue further. Inspecting the simple distribution of lenght of the response, can also tell you about potential artifacts in your data collection that you should look into. Maybe there are more than one template important for your parsing, or there is a certain standard response given when data is missing.
When presenting simple statistics about the data collection, it provides transparency and signals the commitment you have made as a data curator.

## Exercise Section 10.2: Parsing a Table from HTML using BeautifulSoup.

Yesterday I showed you a neat little prepackaged function in pandas that did all the work. However today we should learn the mechanics of it. *(It is not just for educational purposes, sometimes the package will not do exactly as you want.)*

We hit the Basketball stats page from yesterday again: https://www.basketball-reference.com/leagues/NBA_2018.html.


> **Ex. 10.2.1:** Here we practice simply locating the table node of interest using the `find` method build into BeautifoulSoup. But first we have to fetch the HTML using the `requests` module. Parse the tree using `BeautifulSoup`. And then use the **>Inspector<** tool (* right click on the table < press inspect element *) in your browser to see how to locate the Eastern Conference table node - i.e. the *tag* name of the node, and maybe some defining *attributes*.

In [2]:
## Solution
from bs4 import BeautifulSoup
import requests
url = 'https://www.basketball-reference.com/leagues/NBA_2018.html'
response = connector.get(url,'table_exercise')
html = response[0].text
soup = BeautifulSoup(html,'html.parser')
table_node = soup.find('table',attrs={'id':'confs_standings_E'}) # find table node with specific attribute

You have located the table should now build a function that starts at a "table node" and parses the information, and outputs a pandas DataFrame. 

Inspect the element either within the notebook or through the **>Inspector<** tool and start to see how a table is written in html. Which tag names can be used to locate rows? How will you iterate through columns. Were is the header located?

> **Ex. 10.1.2:** First you parse the header which can be found in the canonical tag name: thead. 
Next you use the `find_all` method to search for the tag, and iterate through each of the elements extracting the text, using the `.text` method builtin to the the node object. Store the header values in a list container. 

> **Ex. 10.1.3:** Next you locate the rows, using the canonical tag name: tbody. And from here you search for all rows tags. Fiugre out the tag name yourself, inspecting the tbody node in python or using the **Inspector**. 

> **Ex. 10.1.4:** Next run through all the rows and extract each value, similar to how you extracted the header. However here is a slight variation: Since each value node can have a different tag depending on whether it is a digit or a string, you should use the `.children` method instead of the `.find_all` - (or write compile a regex that matches both the td tag and the th tag.) 
>Once the value nodes of each row has been located using the `.children` method you should extract the value. Store the extracted rows as a list of lists: ```[[val1,val2,...valk],...]```

In [3]:
import pandas as pd
# Solution
name = table_node.caption.text
# parse header
header = table_node.thead.find_all('th') # locate each column name using the `th` tag, which entail a string .
# extract the label you want. brevity use .text, for a more informative get teh aria-label attribute
header = [col['aria-label'] for col in header]
print(header,len(header))
# parse rows: the canonical tbody locates the data body.
body = table_node.tbody
# rows are found using the "tr" tag
rows = body.find_all('tr')
# each row value has different tags depending on their type (digit or string)
# function to check what tag it is and either convert to float or not. 
import numpy as np
def convert_value_type(value_node):
    if value_node.name == 'th':
        return value_node.text
    else: # assume node is td:
        try: 
            return float(value_node.text)
        except:
            return np.nan # assume there is no value if it fails. 
data = []
for row_node in rows:
    row = []
    for child in row_node.children:
        row.append(convert_value_type(child))
    data.append(row)


['Eastern Conference', 'Wins', 'Losses', 'Win-Loss Percentage', 'Games Behind', 'Points Per Game', 'Opponent Points Per Game', 'Simple Rating System'] 8


In [4]:

df = pd.DataFrame(data,columns=header)
def parse_html_table(table_node):
    # parse header
    header = table_node.thead.find_all('th') # locate each column name using the `th` tag, which entail a string .
    # extract the label you want. brevity use .text, for a more informative get teh aria-label attribute
    header = [col['aria-label'] for col in header]
    # parse rows: the canonical tbody locates the data body.
    body = table_node.tbody
    # rows are found using the "tr" tag
    rows = body.find_all('tr')
    # each row value has different tags depending on their type (digit or string)
    # function to check what tag it is and either convert to float or not. 
    import numpy as np
    def convert_value_type(value_node):
        if value_node.name == 'th':
            return value_node.text
        else: # assume node is td:
            try: 
                return float(value_node.text)
            except:
                return np.nan # assume there is no value if it fails. 
    data = []
    for row_node in rows:
        row = []
        for child in row_node.children:
            row.append(convert_value_type(child))
        data.append(row)
    df = pd.DataFrame(data,columns=header)
    return df
df = parse_html_table(table_node)
df.head()

,Eastern Conference,Wins,Losses,Win-Loss Percentage,Games Behind,Points Per Game,Opponent Points Per Game,Simple Rating System
0,Toronto Raptors* (1),59.0,23.0,0.720,NaN,111.7,103.9,7.29
1,Boston Celtics* (2),55.0,27.0,0.671,4.0,104.0,100.4,3.23
2,Philadelphia 76ers* (3),52.0,30.0,0.634,7.0,109.8,105.3,4.30
3,Cleveland Cavaliers* (4),50.0,32.0,0.610,9.0,110.9,109.9,0.59
4,Indiana Pacers* (5),48.0,34.0,0.585,11.0,105.6,104.2,1.18


> **Ex. 10.2.6:** Now locate all tables from the page, using the `.find_all` method searching for the table tag name. Iterate through the table nodes and apply the function created for parsing html tables. Store each table in a dictionary using the table name as key. The name is found by accessing the id attribute of each table node, using dictionary-style syntax - i.e. `table_node['id']`.

> ** 10.2.extra.:** Compare your results to the pandas implementation. pd.read_html

In [5]:
# Solution
tables = soup.find_all('table') # locate all table nodes
dfs = {}
for table_node in tables:
    name = table_node['id'] # access the id attribute. 
    table = parse_html_table(table_node) # apply parse_table function
    dfs[name] = table # store table in the dictionary

## Exercise Section 10.2: Practicing Regular Expressions.
This exercise is about developing your experience with designing your own regular expressions.

Remember you can always consult the regular expression reference page [here](https://www.regular-expressions.info/refquick.html), if you need to remember or understand a specific symbol. 

You should practice using *"define-inspect-refine-method"* described in the lectures to systematically ***explore*** and ***refine*** your expressions, and save all the patterns tried. You can download the small module that I created to handle this in the following way: 
``` python
import requests
url = 'https://raw.githubusercontent.com/snorreralund/explore_regex/master/explore_regex.py'
response = requests.get(url)
with open('explore_regex.py','w') as f:
    f.write(response.text)
import explore_regex as e_re
```

Remember to start ***broad*** to gain many examples, and iteratively narrow and refine.

We will use a sample of the trustpilot dataset that you practiced collecting yesterday.
You can load it directly into python from the following link: https://raw.githubusercontent.com/snorreralund/scraping_seminar/master/english_review_sample.csv

> **Ex. 10.2.0:** Load the data used in the exercise using the `pd.read_csv` function. (Hint: path to file can be both a url or systempath). 

>Define a variable `sample_string = '\n'.join(df.sample(2000).reviewBody)` as sample of all the reviews that you will practice on.  (Run it once in a while to get a new sample for potential differences).
Imagine we were a company wanting to find the reviews where customers are concerned with the price of a service. They decide to write a regular expression to match all reviews where a currencies and an amount is mentioned. 

> **Ex. 10.2.1:** 
> Write an expression that matches both the dollar-sign (\$) and dollar written literally, and the amount before or after a dollar-sign. Remember that the "$"-sign is a special character in regular expressions. Explore and refine using the explore_pattern function in the package I created called explore_regex. 
```python
import explore_regex as e_re
explore_regex = e_re.Explore_Regex(sample_string) # Initaizlie the Explore regex Class.
explore_regex.explore_pattern(pattern) # Use the .explore_pattern method.
```


Start with exploring the context around digits ("\d") in the data. 

In [6]:
import pandas as pd
import re
import requests
# download data
path2data = 'https://raw.githubusercontent.com/snorreralund/scraping_seminar/master/english_review_sample.csv'
df = pd.read_csv(path2data)
# download module
url = 'https://raw.githubusercontent.com/snorreralund/explore_regex/master/explore_regex.py'
response = requests.get(url)
# write script to your folder to create a locate module
with open('explore_regex.py','w') as f:
    f.write(response.text)
# import local module
import explore_regex as e_re

In [7]:
import re
digit_re = re.compile('[0-9]+') 
df['hasNumber'] = df.reviewBody.apply(lambda x: len(digit_re.findall(x))>0)
sample_string = '\n'.join(df[df['hasNumber']].sample(1000).reviewBody)

In [8]:
explore_regex = e_re.ExploreRegex(sample_string)

In [9]:
pattern = '[£$] ?[0-9]+(?:[.,][0-9]+)?|[0-9]+(?:[.,][0-9]+)? ?(?:USD|usd)|[0-9]+(?:[.,][0-9]+)? ?(?:dollars|DOLLARS)' #[£$] ?[0-9]+(?:[.,][0-9]+)?|[0-9]+(?:[.,][0-9]+)? ?(?:USD|usd)|
#pattern = '[£$] ?[0-9]+(?:[.,][0-9]+)?|[0-9]+(?:[.,][0-9]+)? ?(?:USD|usd)|[0-9]+(?:[.,][0-9]+)? ?(dollars|DOLLARS)' # 

explore_regex.explore_pattern(pattern,context=20)

------ Pattern: [£$] ?[0-9]+(?:[.,][0-9]+)?|[0-9]+(?:[.,][0-9]+)? ?(?:USD|usd)|[0-9]+(?:[.,][0-9]+)? ?(?:dollars|DOLLARS)	 Matched 267 patterns -----
Match: $100	Context:t it’s $300 and the $100 of insurance doesn’
Match: $20.00	Context:yesterday, I put in $20.00 , but, when I tried
Match: $100	Context:shipping and 25 off $100 and that really sav
Match: $10	Context: other sites wanted $10. The only thing was
Match: $25.00	Context:k out. I bought the $25.00 gift cards from Fan
Match: $200	Context:m this cost me over $200.-. They told me aft
Match: $9.98	Context:pany at Walmart for $9.98. Could of saved tim
Match: £78	Context:ed the order on the £78 dress I'd wanted fo
Match: $42	Context:I do, you know that $42/night would not get
Match: $18	Context:the mattress needed $18 straps to hold it. 


> **Ex.10.2.3** Use the .report() method. e_re.report(), and print the all patterns in the development process using the .pattern method - i.e. e_re.patterns 


In [10]:
#[Answer 10.2.3]
explore_regex.report('soft')

------ Pattern: [£$] ?[0-9]+(?:[.,][0-9]+)?|[0-9]+(?:[.,][0-9]+)? ?(?:USD|usd)|[0-9]+(?:[.,][0-9]+)? ?(?:dollars|DOLLARS)	 Matched 267 patterns -----


ValueError: zero-size array to reduction operation minimum which has no identity

<Figure size 864x576 with 0 Axes>

In [11]:
explore_regex.patterns

['[£$] ?[0-9]+(?:[.,][0-9]+)?|[0-9]+(?:[.,][0-9]+)? ?(?:USD|usd)|[0-9]+(?:[.,][0-9]+)? ?(?:dollars|DOLLARS)']

>** Ex. 10.2.4** 
Finally write a function that takes in a string and outputs if there is a match. Use the .match function to see if there is a match (hint if does not return a NoneType object - `re.match(pattern,string)!=None`).

> Define a column 'mention_currency' in the dataframe, by applying the above function to the text column of the dataframe. 
*** You should have approximately 310 reviews that matches. - but less is also alright***

> **Ex. 10.2.5** Explore the relation between reviews mentioning prices and the average rating. 

> **Ex. 10.2.extra** Define a function that outputs the amount mentioned in the review (if more than one the largest), define a new column by applying it to the data, and explore whether reviews mentioning higher prices are worse than others by plotting the amount versus the rating.

In [12]:
#[Answer to 10.2.4-5]


In [13]:
pattern = '[£$] ?[0-9]+(?:[.,][0-9]+)?|[0-9]+(?:[.,][0-9]+)? ?(?:USD|usd)|[0-9]+(?:[.,][0-9]+)? ?(?:dollars|DOLLARS)'
currency_re = re.compile(pattern)
def match_currency(string):
    return len(currency_re.findall(string))>0
df['mention_currency'] = df.reviewBody.apply(match_currency)

In [14]:
df.groupby('mention_currency').reviewRating_ratingValue.mean()

mention_currency
False    4.507275
True     2.935275
Name: reviewRating_ratingValue, dtype: float64

> **Ex. 10.2.5:** Now we write a regular expression to extract emoticons from text.
Start by locating all mouths ')' of emoticons, and develop the variations from there. Remember that paranthesis are special characters in regex, so you should use the escape character.

In [15]:
sample_string = '\n'.join(df.sample(1000).reviewBody)
emoticon_regex = e_re.ExploreRegex(sample_string)

In [16]:
pattern = '[:;][-Oo]?[()D]'
emoticon_regex.explore_pattern(pattern,context=20)

------ Pattern: [:;][-Oo]?[()D]	 Matched 5 patterns -----
Match: :)	Context:in really it worked :)
Fast, hassle-free, 
Match: :)	Context:y with my purchase! :)
Townfair in Johnsto
Match: :)	Context:e with good bundles :)
Planning a trip to 
Match: :)	Context:always profesionals :)
Nice room, nice loc
Match: :)	Context:que ideas possible. :)
Outstanding service


In [17]:
#emoticon_regex.report()

In [18]:
emoticon_regex.pattern2n_match

{'[:;][-Oo]?[()D]': 5}